## Laboratorium 2


### Huffman's static algorithm

In [4]:
import time
from queue import PriorityQueue
import heapq

Algorytm do utworzenia drzewa dla skompresowania danych - Huffman statyczny 

In [ ]:
class StaticHuffmanTree():
    def __init__(self, filename):
        self.root = None
        self.left = None
        self.right = None
        self.filename = filename
        self.text = None
        self.char_freq = dict()

    def createHuffmanTree(self):
        n = len(self.char_freq)
        Q = []
        heapq.heapify(Q, self.char_freq.items())
        for item in self.char_freq.items():
            heapq.heappush(Q, (item[1], item[0]))
    

    def _readAllFile(self):
        with open(self.filename, "r") as f:
            text = f.read()
    
    def _loadChars(self):
        for char in self.text:
            if char in self.char_freq:
                self.char_freq[char] += 1
            else:
                self.char_freq[char] = 1
